In [24]:
import numpy as np
import math

# 0. Load Utilities

In [25]:
Clock = 0

In [26]:
class DTStat():
    # Generic discrete-time statistics object

    def __init__(self):
        # Excecutes when the DTStat object is created to initialize variables
        self.Sum = 0.0
        self.SumSquared = 0.0
        self.NumberOfObservations = 0.0
    
    def Record(self,X):
        # Update the DTStat
        self.Sum = self.Sum + X
        self.SumSquared = self.SumSquared + X * X
        self.NumberOfObservations = self.NumberOfObservations + 1
        
    def Mean(self):
        # Return the sample mean
        mean = 0.0
        if self.NumberOfObservations > 0.0:
            mean = self.Sum / self.NumberOfObservations
        return mean

    def StdDev(self):
        # Return the sample standard deviation
        stddev = 0.0
        if self.NumberOfObservations > 1.0:
            stddev = math.sqrt((self.SumSquared - self.Sum**2 / self.NumberOfObservations) / (self.NumberOfObservations - 1))
        return stddev
            
    def N(self):
        # Return the number of observations collected
        return self.NumberOfObservations
    
    def Clear(self):
        # Clear statistics
        self.Sum = 0.0
        self.SumSquared = 0.0
        self.NumberOfObservations = 0.0

In [27]:
class CTStat():
    # Generic continuous-time statistics object
    # Note that CTStat should be called AFTER the value of the variable changes

    def __init__(self):
        # Excecuted when the CTStat object is created to initialize variables
        self.Area = 0.0
        self.Tlast = 0.0
        self.TClear = 0.0
        self.Xlast = 0.0
        
    def Record(self,X):
        # Update the CTStat from the last time change and keep track of previous value
        self.Area = self.Area + self.Xlast * (Clock - self.Tlast)
        self.Tlast = Clock
        self.Xlast = X

    def Mean(self):
        # Return the sample mean up through the current time but do not update
        mean = 0.0
        if (Clock - self.TClear) > 0.0:
            mean = (self.Area + self.Xlast * (Clock - self.Tlast)) / (Clock - self.TClear)
        return mean
    
    def Clear(self):
        # Clear statistics
        self.Area = 0.0
        self.Tlast = Clock
        self.TClear = Clock

# 1. Parameters for Simulation

In [28]:
meanArrival = 1.25 # mean of arrival time distribution (Exponential)
k = 1 # shape parameter for service time distribution (Gamma)

# 2. Set up stats

In [29]:
Wait = DTStat() # a DTstat for average waiting time of each customer
Idle = DTStat()
# Average queue length will be an attribute of the queue object. No need to define it for now.

# 3. Set up server

In [30]:
class Resource():
    # This is a generic Resource object that also keeps track of statistics
    # on number of busy resources

    def __init__(self):
        # Executes when the resource object is created to initialize variables
        # and add number of busy Resources statistic to TheCTStats list
        self.Busy = 0
        self.NumberOfUnits = 0
        self.NumBusy = CTStat()
        
    def Seize(self, Units):
        # Seize Units of resource then update statistics
        # Returns False and does not seize if not enouge resources available;
        # otherwise returns True
        diff = self.NumberOfUnits - Units - self.Busy
        if diff >= 0:
            # If diff is nonnegative, then there are enough resources to seize
            self.Busy = self.Busy + Units
            self.NumBusy.Record(float(self.Busy))
            seize = True
        else:
            seize = False
        return seize
        
    def Free(self, Units):
        # Frees Units of resource then update statistics
        # Returns False and does not free if attempting to free more resources than available;
        # otherwise returns True
        diff = self.Busy - Units
        # If diff is negative, then trying to free too many resources
        if diff < 0:
            free = False
        else:
            self.Busy = self.Busy - Units
            self.NumBusy.Record(float(self.Busy))
            free = True
        return free
    
    def Mean(self):
        # Returtime-average number of busy resources up to current time
        return self.NumBusy.Mean()
        
    def SetUnits(self, Units):
        # Set the capacity of the resource (number of identical units)
        self.NumberOfUnits = Units
        
    def Inspect(self):
        # return number of free server(s)
        return self.NumberOfUnits - self.Busy

In [31]:
Server = Resource()
Server.SetUnits (1) # only one server in the queue

In [32]:
#==================================================================
# EXERCISE
#==================================================================
# Determine the values of free_1 to free_4 before running this cell

free_1 = Server.Inspect()

Server.Seize(1)
free_2 = Server.Inspect()

Server.Seize(1)
free_3 = Server.Inspect()

Server.Free(1)
free_4 = Server.Inspect()

In [33]:

free_1 = Server.Inspect()
free_1

1

In [34]:
Server.Seize(1)
free_2 = Server.Inspect()
free_2

0

In [35]:
Server.Free(1)
free_4 = Server.Inspect()
free_4

1

In [36]:
[free_1,free_2,free_3,free_4]

[1, 0, 0, 1]

In [37]:
# rerun this cell to reset the server after the exercise
Server = Resource()
Server.SetUnits (3) # only one server in the queue

# 4. Set up queue

In [38]:
class FIFOQueue():
    # This is a generic FIFO Queue object that also keeps track
    # of statistics on the number in the queue (WIP)

    def __init__(self):
        # Executes when the FIFOQueue object is created to add queue statistics
        # to TheCTStats list
        self.WIP = CTStat()
        self.ThisQueue = []
        
    def NumQueue(self):
        # Return current number in the queue
        return len(self.ThisQueue)
        
    def Add(self,X):
        # Add an entity to the end of the queue
        self.ThisQueue.append(X)
        numqueue = self.NumQueue()
        self.WIP.Record(float(numqueue))    
    
    def Remove(self):
        # Remove the first entity from the queue and return the object
        # after updating the queue statistics
        if len(self.ThisQueue) > 0:
            remove = self.ThisQueue.pop(0)
            self.WIP.Record(float(self.NumQueue()))
            return remove
        
    def Mean(self):
        # Return the average number in queue up to the current time
        return self.WIP.Mean()

In [39]:
Queue = FIFOQueue()

In [40]:
#===================================================
# EXERCISE
#===================================================
# Determine the values of variables in the next cell 
# before running the commands.

m_0 = Queue.Mean()

Clock = 1 # at time t=1
Queue.Add('something')
m_1 = Queue.Mean()

Clock = 2 # time t=2
Queue.Remove()
m_2 = Queue.Mean()

Clock = 4
Queue.Add('somethingElse')
m_4 = Queue.Mean()

Clock = 5
m_5 = Queue.Mean()

In [41]:
m_0, m_1, m_2, m_4, m_5, Queue.ThisQueue

(0.0, 0.0, 0.5, 0.25, 0.4, ['somethingElse'])

In [42]:
# run this cell to reset after the exercise
Clock = 0
Queue = FIFOQueue()

# 5. Set up calendar

In [43]:
class EventNotice():
    # This is the generic EventNotice object with EventTime, EventType and 
    # WhichObject attributes
    # Add additional problem-specific attributes here
    def __init__(self):
        self.EventTime = 0.0
        self.EventType = ""
        self.WhichObject = ()

In [44]:
class EventCalendar():
    # This class creates an EventCalendar object which is a list of
    # EventNotices ordered by time. Based on an object created by Steve Roberts.

    def __init__(self):
        self.ThisCalendar = []   
    def Schedule(self,addedEvent):
        # Add EventNotice in EventTime order
        if len(self.ThisCalendar) == 0:  #no events in calendar
            self.ThisCalendar.append(addedEvent)
        elif self.ThisCalendar[-1].EventTime <= addedEvent.EventTime:
            self.ThisCalendar.append(addedEvent)
        else:
            for rep in range(0,len(self.ThisCalendar),1):
                if self.ThisCalendar[rep].EventTime > addedEvent.EventTime:
                    break
            self.ThisCalendar.insert(rep,addedEvent)
    
    def Remove(self):
        # Remove next event and return the EventNotice object
        if len(self.ThisCalendar) > 0:
            return self.ThisCalendar.pop(0)
        
    def N(self):
        # Return current number of events on the event calendar
        return len(self.ThisCalendar)

In [45]:
#===================================================
# EXERCISE
#===================================================
# Determine the values of variables in the next cell 
# before running the commands.
Calendar = EventCalendar()

eventA = EventNotice()
eventA.EventTime = 1.0
eventA.EventType = 'A'

eventB = EventNotice()
eventB.EventTime = 4.5
eventB.EventType = 'B'

eventC = EventNotice()
eventC.EventTime = 3.2
eventC.EventType = 'C'

Calendar.Schedule(eventA)
Calendar.Schedule(eventB)
Calendar.Schedule(eventC)

# Calendar.Remove()

In [46]:
Calendar.N(), [ event.EventType for event in Calendar.ThisCalendar ]

(3, ['A', 'C', 'B'])

In [47]:
# run this cell to reset after the exercise
Calendar = EventCalendar()

# 6. Finish the script by completing the following two functions

In [48]:
class Entity():
    # This is the generic Entity that has a single attribute CreateTime
    def __init__(self):
        # Executes with the Entity object is created to initialize variables
        # Add additional problem-specific attributes here
        self.CreateTime = Clock

In [49]:
def Schedule(calendar,EventType, EventTime):
    #Schedule future events of EventType to occur at time SimClasses.Clock + EventTime
    
    addedEvent = EventNotice()
    addedEvent.EventType = EventType
    addedEvent.EventTime = Clock + EventTime
    # print("SimClasses.Clock is %f" % SimClasses.Clock)
    # print(EventTime)
    calendar.Schedule(addedEvent)

In [50]:
IdleTime = 0

def Arrival():
    global IdleTime
    #========
    # Step 1
    #========
    # Create this newly arrived customer,
    # and send the customer into the queue.
    Customer = Entity() # Remember: Class Entity automatically registers CreateTime as an attribute.
    Queue.Add(Customer)
    
    #========
    # Step 2
    #========
    # Schedule the next arrival: Exponential Dist, meanArrival = 4
    # Note: Only use the following two event types: "Arrival", "EndOfService".
    Schedule(Calendar,EventType = 'Arrival', EventTime = np.random.exponential( meanArrival )) 
    
    
    #========
    # Step 3
    #========
    # Consider a (not so) special case:
    # "What is the right action if this customer arrives when the queue IS EMPTY?"
    # In this case, immediately send the customer to the server.
    if Server.Busy == 0: 
        Server.Seize(1) # If not busy, the server would be occupied now (by calling Seize method).
        Idle.Record(Clock - IdleTime)
        Schedule(Calendar, EventType = 'EndOfService', EventTime = np.random.exponential(k))


In [51]:
def EndOfService():
    global IdleTime
    #========
    # Step 1
    #========
    # First of all, a customer would leave the queue.
    # Enter Your Code Here
    
    #========
    # Step 2
    #========
    # Update Wait (DTstat): we need to record the waiting time for this customer
        # Enter Your Code Here
    
    #========
    # Step 3
    #========
    # Immediately after the service of DepartingCustomer,
    # there are two different scenarios depending on the status of the queue.
    # If the queue is not empty, then the next service would start right away.
    # Otherwise, the server would become idle.
    
    # Enter Your Code Here

# 7. run simulation

In [52]:
# no need to look into the function for now.
# just know that all the stats and components will be reinitialized
# at the beginning of a simulation trial.

def SimFunctionsInit(calendar,queues,ctstats,dtstats,resources):
    # Function to initialize SimFunctions.Python
    # Typically called before the first replication and between replications
    Clock = 0.0
    #Emply event calendar
    while (calendar.N() > 0):
        EV = calendar.Remove()
        
    #Empty queues
    #On first call, append the CStats created by FIFOQueue
    for Q in queues:
        if Q.WIP not in ctstats:
            ctstats.append(Q.WIP)
        while Q.NumQueue() > 0:
            En = Q.Remove()

    #Reinitialize Resources
    #On first call, append the CStats created by FIFOQueue and Resource
    for Re in resources:
        Re.Busy = 0.0
        if Re.NumBusy not in ctstats:
            ctstats.append(Re.NumBusy)  
    
    #Clear statistics
    for CT in ctstats:
        CT.Clear()
        CT.Xlast = 0.0   # added bln 
        
    for DT in dtstats:
        DT.Clear()

In [53]:
# no need to look into this block.
# just some preparations so that SimFunctionsInit defined above can be called.

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

TheDTStats.append(Wait)
TheDTStats.append(Idle)
TheQueues.append(Queue)
TheResources.append(Server)

AllWaitMean = []
AllIdleMean = []

AllQueueMean = []
AllQueueNum = []
AllServerMean = []


In [54]:
# For demonstration purpose, let us fix the random seed at this step.
np.random.seed(2020)

In [55]:
# some parameters for simulation
rep = 10 # number of trials
RunLength = 50000 # stop the trial if Clock reaches 50,000

In [56]:
for _ in range(rep):
    #Clock = 0
    #===============
    # Initialization
    #===============
    SimFunctionsInit(Calendar,TheQueues,TheCTStats,TheDTStats,TheResources)
    
    # Two events need to be scheduled.
    # The first arrival,
    # and EndSimulation (so we know when to stop the trial).
    Schedule(Calendar,"Arrival", np.random.exponential(meanArrival) )
    Schedule(Calendar,"EndSimulation",RunLength)
    
    
    #================
    # Simulation Loop
    #================
    # First, fetch the next event from the calendar.
    NextEvent = Calendar.Remove()
    # Always remember to UPDATE THE CLOCK when jumping to the next event!
    Clock = NextEvent.EventTime
    
    # Then, depending on the type of the event, call the functions we wrote accordingly.
    if NextEvent.EventType == "Arrival":
        Arrival()
    elif NextEvent.EventType == "EndOfService":
        EndOfService() 
    
    # the loop: as long as we have not reached the end of simulation,
    # repeat the procedure above.
    while NextEvent.EventType != "EndSimulation":
        NextEvent = Calendar.Remove()
        Clock = NextEvent.EventTime
        if NextEvent.EventType == "Arrival":
            Arrival()
        elif NextEvent.EventType == "EndOfService":
            EndOfService()
    
    #=============
    # Report Stats
    #=============
    AllWaitMean.append(Wait.Mean())
    AllIdleMean.append(Idle.Mean())
    AllQueueMean.append(Queue.Mean())
    AllQueueNum.append(Queue.NumQueue())
    AllServerMean.append(Server.Mean())

In [57]:
np.mean(AllWaitMean)

5.028407614948223

In [58]:
np.mean(AllQueueMean)

4.042563539479014

In [105]:
np.mean(AllQueueNum)

3.0

In [59]:
np.mean(AllServerMean)

0.8022500312380838

In [107]:
np.mean(AllIdleMean)

1.2417461439525732